In [1]:
import pandas as pd
import requests
import json
import time

# Import API key
from keys import api_key

file_to_load = "Resources/StateAbbr.csv"

#Read purchasing file and store into Pandas DF
state_abbr = pd.read_csv(file_to_load)
state_abbr.head()

,State Abbr
0,AK
1,AL
2,AR
3,AS
4,AZ


In [2]:
# Get MN Data
#api_url = 'https://api.usa.gov/crime/fbi/sapi/api/summarized/state/MN/violent-crime/2017/2018?API_KEY=' + api_key
#response = requests.get(api_url)
  # list of crime dictionary from api to convert into dataFrame

mn_crime_results = []


State_list = ["NY","NC","OR","TX","MN"]
for state in State_list : 
    
    api_url = 'https://api.usa.gov/crime/fbi/sapi/api/summarized/state/' + state + '/violent-crime/2017/2017?API_KEY=' + api_key
    response = requests.get(api_url)
    print(api_url)

    # add to a dataframe
    if response.status_code == 200:
        # call the api to get http://localhost:8888/notebooks/Claudia%20Notebook.ipynb#the total count
        json = response.json()
        total_count = json['pagination']['count']

        # Get number of pages to loop over
        per_page = json['pagination']['per_page']
        total_pages  = json['pagination']['pages']
        print(f'{total_pages} with {per_page} per page')

        # for loop over all pages to accumulate total dataset
        for page in range(0, total_pages):
            # Prevent rate limiting & locked account
            print(f'On page {page}')
            time.sleep(1.1)

            api_page_url = api_url + '&page=' + str(page)
            crime_response = requests.get(api_page_url)
            if crime_response.status_code == 200:
                crime_json = crime_response.json()
                for crime_dictionary in crime_json['results']:
                    mn_crime_results.append(crime_dictionary)
            else:
                print(f"Error getting crime data for page: {page}")
    else:
        print("Error getting crime data")

    print('------- Done Fetching Data -------')

https://api.usa.gov/crime/fbi/sapi/api/summarized/state/NY/violent-crime/2017/2017?API_KEY=LOkNfh4BeifO5u9VzX5u4TLZataPUWuEH9joUoLO
28 with 20 per page
On page 0
On page 1
On page 2
On page 3
On page 4
On page 5
On page 6
On page 7
On page 8
On page 9
On page 10
On page 11
On page 12
On page 13
On page 14
On page 15
On page 16
On page 17
On page 18
On page 19
On page 20
On page 21
On page 22
On page 23
On page 24
On page 25
On page 26
On page 27
------- Done Fetching Data -------
https://api.usa.gov/crime/fbi/sapi/api/summarized/state/NC/violent-crime/2017/2017?API_KEY=LOkNfh4BeifO5u9VzX5u4TLZataPUWuEH9joUoLO
20 with 20 per page
On page 0
On page 1
On page 2
On page 3
On page 4
On page 5
On page 6
On page 7
On page 8
On page 9
On page 10
On page 11
On page 12
On page 13
On page 14
On page 15
On page 16
On page 17
On page 18
On page 19
------- Done Fetching Data -------
https://api.usa.gov/crime/fbi/sapi/api/summarized/state/OR/violent-crime/2017/2017?API_KEY=LOkNfh4BeifO5u9VzX5u4TLZata

In [3]:
mn_crime_df = pd.DataFrame(mn_crime_results)
mn_crime_df.head()

,actual,cleared,data_year,offense,ori,state_abbr
0,58,36,2017,violent-crime,NY0030000,NY
1,362,147,2017,violent-crime,NY0030100,NY
2,74,44,2017,violent-crime,NY0030200,NY
3,91,49,2017,violent-crime,NY0030300,NY
4,0,0,2017,violent-crime,NY0032000,NY


In [4]:
data = {'State': ['MN'], 'Offenses': [len(mn_crime_df)]}
mn_count =  pd.DataFrame.from_dict(data)
mn_count.head()

,State,Offenses
0,MN,2520


In [5]:
mn_crime_df 
crime_byState = mn_crime_df.groupby(["state_abbr", "data_year"])

CrimeData_df = pd.DataFrame({"Crime Count" : crime_byState["actual"].sum()})
CrimeData_df.head()

,,Crime Count
state_abbr,data_year,
MN,2017,15048
NC,2017,40660
NY,2017,123481
OR,2017,12495
TX,2017,121723


In [6]:
CrimeData_df.reset_index(inplace=True)
 
CrimeData_df.rename(columns={"state_abbr":"state","data_year":"year"},inplace=True)
CrimeData_df

,state,year,Crime Count
0,MN,2017,15048
1,NC,2017,40660
2,NY,2017,123481
3,OR,2017,12495
4,TX,2017,121723


In [7]:
CrimeData_df.to_csv("Resources/CrimeData.csv", index=False, header=True)